In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

def search_videos(query: str, size: int = 5) -> list[dict]:
    """
    Search for videos whose titles or subtitles match a given query.
    
    Returns highlighted match information including video IDs and snippets.

    Args:
        query (str): The search query string to match against video titles and subtitles. Must be a non-empty string.
        size (int, optional): Maximum number of results to return. Must be a positive integer. Defaults to 5.
    """
    body = {
        "size": size,
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["title^3", "subtitles"],
                "type": "best_fields",
                "analyzer": "english_with_stop_and_stem"
            }
        },
        "highlight": {
            "pre_tags": ["*"],
            "post_tags": ["*"],
            "fields": {
                "title": {
                    "fragment_size": 150,
                    "number_of_fragments": 1
                },
                "subtitles": {
                    "fragment_size": 150,
                    "number_of_fragments": 1
                }
            }
        }
    }
    
    response = es.search(index="podcasts", body=body)
    hits = response.body['hits']['hits']
    
    results = []
    for hit in hits:
        highlight = hit['highlight']
        highlight['video_id'] = hit['_id']
        results.append(highlight)
    
    return results

def get_subtitles_by_id(video_id: str) -> dict:
    """
    Retrieve the full subtitle content for a specific video.

    Args:
        video_id (str): the YouTube video id for which we want to get the subtitles
    """
    result = es.get(index="podcasts", id=video_id)
    return result['_source']

In [8]:
from pydantic_ai.messages import FunctionToolCallEvent

class NamedCallback:
    """Stream handler that prints the tool calls triggered by an agent."""
    
    def __init__(self, agent: Agent):
        self.agent_name = agent.name
    
    async def _print_function_calls(self, ctx, event) -> None:
        # Detect nested streams
        if hasattr(event, "__aiter__"):
            async for sub_event in event:
                await self._print_function_calls(ctx, sub_event)
            return
        
        if isinstance(event, FunctionToolCallEvent):
            tool_name = event.part.tool_name
            args = event.part.args
            print(f"TOOL CALL ({self.agent_name}): {tool_name}({args})")
    
    async def __call__(self, ctx, event) -> None:
        await self._print_function_calls(ctx, event)

In [19]:
from pydantic_ai import Agent

research_instructions = """
You are an autonomous research agent. Your goal is to perform deep, multi-stage research on the
given topic using the available search function.

Research process:

Stage 1: Initial Exploration  
- Using your own knowledge of the topic, perform 3-5 broad search queries to understand the main topic
  and identify related areas. Only use search function.
- After the initial search exploration, summarize key concepts, definitions, and major themes.
- You MUST inspect the full transcript to be able to provide a better write up for the user.

Stage 2: Deep Investigation 
- Perform 5-6 refined queries focusing on depth.
- Inspect relevant documents for specific mechanisms, case studies, and technical details.
- Gather diverse viewpoints and data to strengthen depth and accuracy.

Rules:
1. Search queries:
   - Do not include years unless explicitly requested by the user.
   - Use timeless, concept-based queries (e.g., "AI investment trends" not "AI trends 2024").

2. The article must contain:
   - A clear, descriptive title.
   - An introduction with 2-3 paragraphs (each 4-6 sentences).
   - At least 10 sections, each focused on a distinct subtopic.
   - Each section must have at least 4 paragraphs, preferably 5-6.

3. Each paragraph must have at least one reference object containing:
   - video_id (YouTube video ID)
   - timestamp ("mm:ss" or "h:mm:ss")
   - quote (a short excerpt from that video segment)

4. Evidence quality:
   - All claims must be traceable to real YouTube sources.
   - References must correspond to valid video_id and timestamp pairs.
   - Do not fabricate data or sources.

5. Tone and style:
   - Write in a professional, neutral, analytical tone.
   - Emphasize explanation, synthesis, and comparison.

6. Output format: Markdown
""".strip()

research_agent = Agent(
    name='research_agent',
    instructions=research_instructions,
    model='openai:gpt-5-mini',
    tools=[search_videos, get_subtitles_by_id]
)

In [17]:
research_callback = NamedCallback(research_agent)

In [18]:
result = await research_agent.run(
    user_prompt='how do I get rich with AI?',
    event_stream_handler=research_callback
)

print(result.output)


TOOL CALL (research_agent): search_videos({"query":"making money with AI","size":5})
TOOL CALL (research_agent): search_videos({"query":"AI startup business model monetization","size":5})
TOOL CALL (research_agent): search_videos({"query":"ways to make money with AI","size":5})
TOOL CALL (research_agent): search_videos({"query":"AI business models YouTube","size":8})
TOOL CALL (research_agent): search_videos({"query":"AI monetization strategies YouTube","size":8})
TOOL CALL (research_agent): get_subtitles_by_id({"video_id": "xCjzA_8S4kI"})
TOOL CALL (research_agent): get_subtitles_by_id({"video_id": "t3HDdVWQzNM"})
TOOL CALL (research_agent): get_subtitles_by_id({"video_id": "aHdaIwOEI8Q"})
TOOL CALL (research_agent): get_subtitles_by_id({"video_id": "_xr1_xb736E"})
TOOL CALL (research_agent): get_subtitles_by_id({"video_id": "EQcY83VA0Us"})
TOOL CALL (research_agent): get_subtitles_by_id({"video_id": "B76J4QkZPWs"})
TOOL CALL (research_agent): get_subtitles_by_id({"video_id": "DX9c__a

In [20]:
summarization_instructions = """
Your task is to summarize the provided YouTube transcript for a specific topic.

Select the parts of the transcripts that are relevant for the topic and search queries.

Format: 
paragraph with discussion (timestamp)
""".strip()

summarization_agent = Agent(
    name='summarization',
    instructions=summarization_instructions,
    model='openai:gpt-4o-mini'
)


In [21]:
user_query = 'how do I get rich with AI?'
search_queries = [
    "investment opportunities in AI",
    "starting AI-focused businesses",
    "AI applications in wealth generation"
]

subtitles = get_subtitles_by_id('1aMuynlLM3o')['subtitles']

prompt = f"""
user query:
{user_query}

search engine queries: 
{'\n'.join(search_queries)}

subtitles:
{subtitles}
""".strip()

summary_result = await summarization_agent.run(prompt)
print(summary_result.output)


To build wealth using AI, one can explore several investment opportunities and business ventures. During the discussion, the speaker emphasizes how leveraging AI in various areas can significantly reduce operational costs and enhance efficiency, thus paving the way for financial success (7:00).

Entrepreneurs can consider starting AI-focused businesses by identifying gaps in existing markets. For instance, developing tools that simplify AI infrastructure can attract clients looking to reduce their expenses related to cloud computing and machine learning (8:30). There are also potential investment opportunities in AI companies, especially those focused on delivering solutions for specific industries, such as finance or healthcare, where efficiency and cost control are crucial (20:12).

Furthermore, innovators in the AI space are encouraged to think beyond traditional methods. For example, to improve profitability, small and medium-sized businesses may focus on customizing pre-existing A

In [22]:

from pydantic_ai import RunContext
import textwrap
import json

async def summarize(ctx: RunContext, video_id: str) -> str:
    """
    Generate a summary for a video based on the conversation history,
    search queries, and the video's subtitles.
    """
    user_queries = []
    search_queries = []
    
    # Extract context from conversation history
    for m in ctx.messages:
        for p in m.parts:
            kind = p.part_kind
            if kind == 'user-prompt':
                user_queries.append(p.content)
            if kind == 'tool-call':
                if p.tool_name == 'search_videos':
                    args = json.loads(p.args)
                    query = args['query']
                    search_queries.append(query)
    
    # Get full subtitles
    subtitles = get_subtitles_by_id(video_id)['subtitles']
    
    # Build contextual prompt
    prompt = textwrap.dedent(f"""
        user query:
        {'\n'.join(user_queries)}
        
        search engine queries: 
        {'\n'.join(search_queries)}
        
        subtitles:
        {subtitles}
    """).strip()
    
    summary_result = await summarization_agent.run(prompt) 
    return summary_result.output

In [23]:
research_agent = Agent(
    name='research_agent',
    instructions=research_instructions,
    model='openai:gpt-4o-mini',
    tools=[search_videos, summarize]  # Replace get_subtitles_by_id with summarize
)



In [25]:
result = await research_agent.run(
    user_prompt='how do I get rich with AI?',
    event_stream_handler=research_callback
)

print(result.output)


TOOL CALL (research_agent): search_videos({"query":"how to get rich with AI"})
TOOL CALL (research_agent): summarize({"video_id": "t3HDdVWQzNM"})
TOOL CALL (research_agent): summarize({"video_id": "_xr1_xb736E"})
TOOL CALL (research_agent): summarize({"video_id": "EQcY83VA0Us"})
TOOL CALL (research_agent): summarize({"video_id": "whpkDmVVGUE"})
TOOL CALL (research_agent): summarize({"video_id": "BP6w_vKySN0"})
# How to Get Rich with AI: Strategies and Insights

In recent years, artificial intelligence (AI) has transformed various sectors, creating new avenues for wealth generation. With the right knowledge, skills, and strategic approaches, individuals can leverage AI technologies to achieve financial success. This comprehensive exploration considers several pathways to wealth through AI, including career development, entrepreneurship, and the application of data-centric methodologies.

## The Shift to Data-Centric AI

The conversation around how to get rich with AI often starts with u

In [6]:
messages = result.new_messages()

for m in messages:
    for p in m.parts:
        kind = p.part_kind
        if kind == 'user-prompt':
            print('USER:', p.content)
            print()
        if kind == 'text':
            print('ASSISTANT:', p.content)
            print()
        if kind == 'tool-call':
            print('TOOL CALL:', p.tool_name, p.args)

USER: how do I get rich with AI?

ASSISTANT: Getting rich with AI involves several strategies, and your approach may depend on your skills, resources, and interests. Here are some avenues to consider:

1. **Develop AI Products**: Create software or applications that leverage AI technologies to solve specific problems. This could be in industries like healthcare, finance, or e-commerce.

2. **Invest in AI Stocks**: Consider investing in companies that are heavily focused on AI, such as major tech firms or innovative startups.

3. **Start an AI Consulting Business**: Offer your expertise in AI to businesses looking to implement AI solutions. This could involve project-based work or long-term partnerships.

4. **Create AI Content**: Share your knowledge about AI through blogs, YouTube channels, or online courses. Monetize your content through ads, sponsorships, or course sales.

5. **Develop AI-Enhanced Services**: If you provide a service (like marketing, customer support, etc.), incorpo